In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM
### The ORM helps us keep our systems decoupled.
### remember that your references will be to classes in your code instead of specific tables in the database, and that we'll be able to influence each class independently

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

## Database setup

In [ ]:
# reflect an existing database into a new model.
# to connect to our SQLite database, we need to use the create_engine() 
# function. This function doesn't actually connect to our database; it just 
# prepares the database file to be connected to later on.
database_path = "../../resources/hawaii.sqlite"
engine = create_engine(f"sqlite:///{database_path}")
# reflect the tables into a new model
Base = automap_base()
Base.prepare(engine, reflect=True)


    # We can view all of the classes that automap found
# def get_table_references( sqlite_db):
#     print(sqlite_db)
#     con = sqlite3.connect(sqlite_db)

#     cursor = con.cursor()
#     sql = f"select name from sqlite_master where type='table';"
#     table_list = cursor.execute(sql).fetchall()

#     cursor.close()
#     return table_list

# def build_table_references(arg):
#     return object

# base = prepare_db_for_connection("hawaii.sqlite")
# print(base)
# for a_class in base.classes:
#     print(a_class.name)
# table_list = []
# table_list = get_table_references("../resources/hawaii.sqlite")
# Metadata().tables.keys
# for table in table_list:
#     print(table[0])
#     base.classes.table[0]
# str.casefold(table_list(0))

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
# to start a communication between the database and the python code
session = Session(engine)
result = session.query(Measurement.date, Measurement.prcp).all()
# result

# Exploratory Climate Analysis

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the last data point in the database. 

# Calculate the date one year from the last date in data set.
previous_year = dt.date(2017,8,23) - dt.timedelta(days=365)
previous_year
# Perform a query to retrieve the data and precipitation scores
results= []
results = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= previous_year).all()
# print(results)

#### 9.2.2 Save Query Results > df = pd.DataFrame(results, columns=['date','precipitation'])
# Save the query results as a Pandas DataFrame and set the index to the date column
precipitation_df = pd.DataFrame(results, columns=["date", "precipitation"])

# Sort the dataframe by date
precipitation_df.reset_index()
precipitation_df.set_index(precipitation_df['date'], inplace=True)

#### 9.2.3 Sort the DataFrame > df = df.sort_index()
precipitation_df.sort_index
# print(precipitation_df.head(25).to_string(index=False))
# Use Pandas Plotting with Matplotlib to plot the data
precipitation_df.plot()


In [ ]:
#### 9.2.5 Generate the Summary
# Use Pandas to calcualte the summary statistics for the precipitation data
precipitation_df.describe()

In [ ]:
# How many stations are available in this dataset?
active_stations = session.query(func.count(Station.station)).all()
# print(active_stations)
# precipitation_df.isna()

In [ ]:
# What are the most active stations?
# List the stations and the counts in descending order.
most_active_stations = session.query(Measurement.station, func.count(Measurement.station)).\
group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).all()
most_active_stations_filtered = []

for station in most_active_stations:
    most_active_stations_filtered.append(station[0])
most_active_stations_filtered
# most_active_stations

In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature most active station?
session.query(func.min(Measurement.tobs), func.max(Measurement.tobs), func.avg(Measurement.tobs)).all()

In [ ]:
# Choose the station with the highest number of temperature observations.
# session.query(Markers).filter(Markers.owner.in_([f.friend_id for f in friends)]).all()
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
from tqdm import tqdm
# print(most_active_stations_filtered)
temp_by_stations = []
for active_station in tqdm(most_active_stations_filtered):
    temp_by_stations.append(session.query(func.min(Measurement.tobs), func.max(Measurement.tobs), func.avg(Measurement.tobs)).filter(Measurement.station == active_station).all())
temp_by_stations

In [ ]:
# Write a function called `calc_temps` that will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
final_temp_obs = session.query(Measurement.tobs).\
filter(Measurement.station == 'USC00519281').\
filter(Measurement.date >= previous_year).all()


# Challenge

In [ ]:
#### convert temperature observation results to a dataframe
temp_observations_df = pd.DataFrame(final_temp_obs, columns=['tobs'])
temp_observations_df

In [ ]:
import matplotlib.pyplot as plt
# the 12 "bins" are visualized in this plot, just like you specified with your code df.plot.hist(bins=12).
# "Bin" refers to each rectangular column in the plot
temp_observations_df.plot.hist(bins=12)